In [1]:
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from imutils import paths
import pickle
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from skimage import filters

c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [14]:
def openfile(file):
    f = open(file,"r", encoding='utf-8')
    lines = f.readlines()

    image_paths=[]  
    image_classes=[]
    for line in lines:
        curLine=line.strip().split(" ")
        image_paths.append(curLine[0]) 
        image_classes.append(curLine[-1])

    time=1
    allt=len(image_paths)
    ims = []
    print("loading images")
    for image_path in image_paths:
        img  = cv2.imread(image_path)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(64,64),interpolation=cv2.INTER_CUBIC)
        ims.append(img)
        print(str(time)+"/"+str(allt) +"  " + image_path, end="\r")
        time = time +1
    print("\nDone!")
    return(ims,image_paths,image_classes)


def siftfeature(image_paths,image_classes,ims):
    sift= cv2.xfeatures2d.SIFT_create()
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        kp, des = sift.detectAndCompute(im,None)
        if str(des) != 'None':
            des_list.append((image_paths[time-1], des))       
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    print("image_classes:%d",len(image_classes))
    print("image_paths:%d",len(image_paths))
    
    time=1
    allt=len(des_list)
    descriptors = des_list[0][1]
    for image_path, descriptor in des_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
        print(str(time) + "/" + str(allt), end="\r")
        time = time +1
    
    k = 128
    voc, variance = kmeans(descriptors, k, 1)
    
    im_features = np.zeros((len(image_paths), k), "float32")
    for i in range(len(image_paths)):
        print("Calculating distance for image "+str(i)+"...", end="\r")
        words, distance = vq(des_list[i][1],voc)
        for w in words:
            im_features[i][w] += 1
            
    nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')
    
    stdSlr = StandardScaler().fit(im_features)
    im_features = stdSlr.transform(im_features)
    
    return(im_features,image_classes)

def lbpfeature(image_paths,image_classes,ims):
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        lbp=local_binary_pattern(im, 8, 1, 'default')
        max_bins = int(lbp.max() + 1)
        des, _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))
        if str(des) != 'None':
            des_list.append(des)
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    
    return(des_list,image_classes)

def hogfeature(image_paths,image_classes,ims):
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        hog_image,des = hog(im, orientations=4, pixels_per_cell=(8,8),cells_per_block=(2, 2),block_norm= 'L2',visualise=True)
        if str(des) != 'None':
            des_list.append((image_paths[time-1], des))
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    print("image_classes:%d",len(image_classes))
    print("image_paths:%d",len(image_paths))
    
    time=1
    allt=len(des_list)
    descriptors = des_list[0][1]
    for image_path, descriptor in des_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
        print(str(time) + "/" + str(allt), end="\r")
        time = time +1
    
    k = 128
    voc, variance = kmeans(descriptors, k, 1)
    
    im_features = np.zeros((len(image_paths), k), "float32")
    for i in range(len(image_paths)):
        print("Calculating distance for image "+str(i)+"...", end="\r")
        words, distance = vq(des_list[i][1],voc)
        for w in words:
            im_features[i][w] += 1
            
    nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')
    
    stdSlr = StandardScaler().fit(im_features)
    im_features = stdSlr.transform(im_features)
    
    return(im_features,image_classes)

def sobelfeature(image_paths,image_classes,ims):
    des_list = []
    time=1
    allt=len(image_paths)
    print("Calculating all the descriptors and keypoints...")
    for im in ims:
        des=filters.sobel(im)
        if str(des) != 'None':
            des=des.flatten()
            des_list.append(des)
            print(str(time) + "/" + str(allt), end="\r")
            time = time +1
        else:
            print(image_paths[time-1])
            del image_classes[time-1]
            del image_paths[time-1]
    print("\nDone!")
    
    return(des_list,image_classes)
    
def siftSVC(im_features,image_classes,val_features):
    print("start sift training model")
    siftclf = LinearSVC()
    siftclf.fit(im_features, np.array(image_classes))
    predictions=siftclf.predict(val_features)
    
    return(siftclf,predictions)

def lbpSVC(im_features,image_classes,val_features):
    print("start lbp training model")
    lbpclf = LinearSVC()
    lbpclf.fit(im_features, np.array(image_classes))
    predictions=lbpclf.predict(val_features)
    
    return(lbpclf,predictions)

def hogSVC(im_features,image_classes,val_features):
    print("start hog training model")
    hogclf = LinearSVC()
    hogclf.fit(im_features, np.array(image_classes))
    predictions=hogclf.predict(val_features)
    
    return(hogclf,predictions)

def sobelSVC(im_features,image_classes,val_features):
    print("start sobel training model")
    sobelclf = LinearSVC()
    sobelclf.fit(im_features, np.array(image_classes))
    predictions=sobelclf.predict(val_features)
    
    return(sobelclf,predictions)

In [3]:
#sift train
print("start sift train")
ims=[]
image_paths=[]
image_classes=[]
ims,image_paths,image_classes=openfile("train.txt")
sift_features,sift_image_classes=siftfeature(image_paths,image_classes,ims)

#sift val
print("start sift val")
imsval=[]
image_pathsval=[]
image_classesval=[]
imsval,image_pathsval,image_classesval=openfile("val.txt")
val_sift_features,sift_fimage_classesval=siftfeature(image_pathsval,image_classesval,imsval)
siftclf,siftpredictions=siftSVC(sift_features,sift_image_classes,val_sift_features)
joblib.dump(siftclf, "sift.pkl", compress=3)
print("end sift train")

start sift train
loading images
63325/63325  images/n02172182/n02172182_9821.JPEGGG
Done!
Calculating all the descriptors and keypoints...
images/n02115641/n02115641_232.JPEG
images/n02117135/n02117135_987.JPEG
images/n02123159/n02123159_2489.JPEG
images/n02129165/n02129165_8099.JPEG
images/n02134084/n02134084_17358.JPEG
images/n02134084/n02134084_829.JPEG
images/n02137549/n02137549_6460.JPEG
images/n02165105/n02165105_510.JPEG
63317/63325
Done!
image_classes:%d 63317
image_paths:%d 63317
start sift valstance for image 63316...
loading images
450/450  images/n02172182/n02172182_9982.JPEG
Done!
Calculating all the descriptors and keypoints...
450/450
Done!
image_classes:%d 450
image_paths:%d 450
start sift training modelimage 449...
end sift train


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [11]:
#lbp train
print("start lbp train")
ims=[]
image_paths=[]
image_classes=[]
ims,image_paths,image_classes=openfile("train.txt")
lbp_features,lbp_image_classes=lbpfeature(image_paths,image_classes,ims)
pca=PCA(n_components=128)
lbp_features=pca.fit_transform(lbp_features)

#lbp val
print("start lbp val")
imsval=[]
image_pathsval=[]
image_classesval=[]
imsval,image_pathsval,image_classesval=openfile("val.txt")
val_lbp_features,lbp_image_classesval=lbpfeature(image_pathsval,image_classesval,imsval)
pca=PCA(n_components=128)
val_lbp_features=pca.fit_transform(val_lbp_features)
lbpclf,lbppredictions=lbpSVC(lbp_features,lbp_image_classes,val_lbp_features)
joblib.dump(lbpclf, "lbp.pkl", compress=3)
print("end lbp train")

start lbp train
loading images
63325/63325  images/n02172182/n02172182_9821.JPEGGG
Done!
Calculating all the descriptors and keypoints...


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:81: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


63325/63325
Done!
start lbp val
loading images
450/450  images/n02172182/n02172182_9982.JPEG
Done!
Calculating all the descriptors and keypoints...


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:81: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


450/450
Done!
start lbp training model
end lbp train


In [12]:
#hog train
print("start hog train")
ims=[]
image_paths=[]
image_classes=[]
ims,image_paths,image_classes=openfile("train.txt")
hog_features,hog_image_classes=hogfeature(image_paths,image_classes,ims)

#hog val
print("start hog val")
imsval=[]
image_pathsval=[]
image_classesval=[]
imsval,image_pathsval,image_classesval=openfile("val.txt")
val_hog_features,hog_image_classesval=hogfeature(image_pathsval,image_classesval,imsval)
hogclf,hogpredictions=hogSVC(hog_features,hog_image_classes,val_hog_features)
joblib.dump(hogclf, "hog.pkl", compress=3)
print("end hog train")

start hog train
loading images
63325/63325  images/n02172182/n02172182_9821.JPEGGG
Done!
Calculating all the descriptors and keypoints...


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\skimage\feature\_hog.py:239: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


63325/63325
Done!
image_classes:%d 63325
image_paths:%d 63325
start hog valistance for image 63324...
loading images
450/450  images/n02172182/n02172182_9982.JPEG
Done!
Calculating all the descriptors and keypoints...


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\skimage\feature\_hog.py:239: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


450/450
Done!
image_classes:%d 450
image_paths:%d 450
start sift training modelimage 449...
end hog train


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [15]:
#sobel train
print("start sobel train")
ims=[]
image_paths=[]
image_classes=[]
ims,image_paths,image_classes=openfile("train.txt")
sobel_features,sobel_image_classes=sobelfeature(image_paths,image_classes,ims)
pca=PCA(n_components=128)
sobel_features=pca.fit_transform(sobel_features)

#sobel val
print("start sobel val")
imsval=[]
image_pathsval=[]
image_classesval=[]
imsval,image_pathsval,image_classesval=openfile("val.txt")
val_sobel_features,sobel_image_classesval=sobelfeature(image_pathsval,image_classesval,imsval)
pca=PCA(n_components=128)
val_sobel_features=pca.fit_transform(val_sobel_features)
sobelclf,sobelpredictions=sobelSVC(sobel_features,sobel_image_classes,val_sobel_features)
joblib.dump(sobelclf, "sobel.pkl", compress=3)
print("end sobel train")

start sobel train
loading images
63325/63325  images/n02172182/n02172182_9821.JPEGGG
Done!
Calculating all the descriptors and keypoints...
63325/63325
Done!
start sobel val
loading images
450/450  images/n02172182/n02172182_9982.JPEG
Done!
Calculating all the descriptors and keypoints...
450/450
Done!
start sobel training model
end sobel train


c:\users\user\anaconda3\envs\tensorflow1\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [17]:
acc_sift=accuracy_score(sift_fimage_classesval, siftpredictions)
acc_lbp=accuracy_score(lbp_image_classesval, lbppredictions)
acc_hog=accuracy_score(hog_image_classesval, hogpredictions)
acc_sobel=accuracy_score(sobel_image_classesval, sobelpredictions)

In [21]:
print(round((acc_sift),3),round((acc_lbp)),round((acc_hog),3),round((acc_sobel),3))

acc_sift acc_lbp acc_hog acc_sobel
0.011 0.0 0.033 0.027


In [ ]:
#joblib.dump((clf, sift_image_paths, stdSlr, k, voc), "sift.pkl", compress=3)